In [ ]:
!pip install pandas numpy matplotlib earthaccess netcdf4 scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 99.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.5.1 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_ma

#imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
import earthaccess
import netCDF4 as nc
import os
from pathlib import Path
from scipy.interpolate import griddata
import pickle
import xml.etree.ElementTree as ET  # NEW: for XML config parsing
from tqdm import tqdm               # NEW: for visual progress bars
import pyproj                       # NEW: for coordinate reprojection if needed

warnings.filterwarnings('ignore')


#cube configurations, more modalities added and changes the res to 1km

In [ ]:
# Define the default 7 modalities used in the HABNet pipeline
HABNET_MODALITIES = [
    'chlor_a',
    'Rrs_412',
    'Rrs_443',
    'Rrs_488',
    'Rrs_531',
    'Rrs_555',
    'par'
]

# Datacube configuration updated to match the HAB paper specs
DATACUBE_CONFIG = {
    'spatial_extent_km': 100,          # 100km area
    'temporal_extent_days': 10,       # 10-day window
    'spatial_resolution_km': 1,       #1km resolution
    'temporal_resolution_days': 1     # Daily granularity
}

# Optional: Define the Gulf of Mexico bounds for event filtering or visualization
GULF_BOUNDS = {
    'lat_min': 24.0,
    'lat_max': 30.5,
    'lon_min': -88.0,
    'lon_max': -80.0
}


#directory setup

In [ ]:
# create directory structure for data storage
def setup_directories():
    base_dir = Path("habnet_data")  # renamed to reflect full pipeline run
    raw_dir = base_dir / "raw_modis_l2"
    processed_dir = base_dir / "processed_datacubes"

    for directory in [base_dir, raw_dir, processed_dir]:
        directory.mkdir(exist_ok=True)

    return base_dir, raw_dir, processed_dir


# get lat/lon around event
def calculate_spatial_bounds(lat, lon, extent_km=100):  # now 100 km
    extent_deg = extent_km / 111.0  # 1 degree ≈ 111 km
    return {
        'lat_min': lat - extent_deg / 2,
        'lat_max': lat + extent_deg / 2,
        'lon_min': lon - extent_deg / 2,
        'lon_max': lon + extent_deg / 2
    }


# get temporal bounds: 9 days before + event day = 10 days total
def calculate_temporal_bounds(event_date, days_before=9):  # changed to 9 days
    end_date = event_date + timedelta(days=1)  # inclusive of event date
    start_date = event_date - timedelta(days=days_before)
    return {'start_date': start_date, 'end_date': end_date}


In [ ]:
def load_and_filter_hab_events(csv_file='habsos_20240430.csv'):
    print("Loading and filtering ground truth HAB events...")

    df = pd.read_csv(csv_file)
    df['SAMPLE_DATE'] = pd.to_datetime(df['SAMPLE_DATE'])

    # Step 1: Filter for Karenia brevis species only
    kb_data = df[df['SPECIES'] == 'brevis'].copy()

    # Step 2: Drop rows missing required data
    kb_clean = kb_data.dropna(subset=['LATITUDE', 'LONGITUDE', 'SAMPLE_DATE', 'CELLCOUNT']).copy()

    # Step 3: Label HAB vs non-HAB based on cell counts
    positive_events = kb_clean[kb_clean['CELLCOUNT'] > 50000].copy()
    positive_events['HAB_EVENT'] = 1

    negative_events = kb_clean[kb_clean['CELLCOUNT'] == 0].copy()
    negative_events['HAB_EVENT'] = 0

    hab_events = pd.concat([positive_events, negative_events], ignore_index=True)

    # Step 4: Filter to Gulf of Mexico region
    gulf_events = hab_events[
        (hab_events['LATITUDE'] >= GULF_BOUNDS['lat_min']) &
        (hab_events['LATITUDE'] <= GULF_BOUNDS['lat_max']) &
        (hab_events['LONGITUDE'] >= GULF_BOUNDS['lon_min']) &
        (hab_events['LONGITUDE'] <= GULF_BOUNDS['lon_max'])
    ].copy()

    # Step 5: Filter to MODIS date range
    modis_start = datetime(2003, 1, 1)
    modis_end = datetime(2018, 12, 31)
    final_events = gulf_events[
        (gulf_events['SAMPLE_DATE'] >= modis_start) &
        (gulf_events['SAMPLE_DATE'] <= modis_end)
    ].copy()

    # Step 6: Assign stable event ID for tracking
    final_events['STABLE_EVENT_ID'] = (
        final_events['SAMPLE_DATE'].dt.strftime('%Y%m%d') + '_' +
        final_events['LATITUDE'].round(4).astype(str) + '_' +
        final_events['LONGITUDE'].round(4).astype(str) + '_' +
        final_events['CELLCOUNT'].astype(int).astype(str)
    )

    # Step 7: Remove duplicates
    final_events = final_events.drop_duplicates(subset=['STABLE_EVENT_ID']).copy()

    # Step 8: Sort chronologically
    final_events = final_events.sort_values('SAMPLE_DATE').reset_index(drop=True)

    print(f"Total filtered HAB event candidates: {len(final_events):,}")
    print(f" - HAB events: {len(final_events[final_events['HAB_EVENT'] == 1]):,}")
    print(f" - Non-HAB events: {len(final_events[final_events['HAB_EVENT'] == 0]):,}")
    print(f" - Date range: {final_events['SAMPLE_DATE'].min().date()} to {final_events['SAMPLE_DATE'].max().date()}")

    return final_events


In [ ]:
def create_mvp_sample(events_df, n_events=5):
    print(f"\nSampling {n_events} MVP events...")

    # Limit to most recent data years (optional: 2015–2018) for MVP
    recent_events = events_df[events_df['SAMPLE_DATE'].dt.year >= 2015].copy()

    # Class balance: half HAB, half non-HAB
    n_positive = min(n_events // 2, len(recent_events[recent_events['HAB_EVENT'] == 1]))
    n_negative = min(n_events // 2, len(recent_events[recent_events['HAB_EVENT'] == 0]))

    positive_sample = recent_events[recent_events['HAB_EVENT'] == 1].sample(
        n=n_positive, random_state=42
    )
    negative_sample = recent_events[recent_events['HAB_EVENT'] == 0].sample(
        n=n_negative, random_state=42
    )

    mvp_sample = pd.concat([positive_sample, negative_sample], ignore_index=True)
    mvp_sample = mvp_sample.sort_values('SAMPLE_DATE').reset_index(drop=True)

    print(f"Sample complete: {len(mvp_sample)} events")
    print(f" - Positive (HAB): {len(mvp_sample[mvp_sample['HAB_EVENT'] == 1])}")
    print(f" - Negative (non-HAB): {len(mvp_sample[mvp_sample['HAB_EVENT'] == 0])}")
    print(f" - Range: {mvp_sample['SAMPLE_DATE'].min().date()} to {mvp_sample['SAMPLE_DATE'].max().date()}")

    return mvp_sample


In [ ]:
# Search MODIS L2 Ocean Color granules for a given date and bounding box
def search_modis_l2_data(date, spatial_bounds):
    bbox = (
        spatial_bounds['lon_min'], spatial_bounds['lat_min'],
        spatial_bounds['lon_max'], spatial_bounds['lat_max']
    )
    try:
        granules = earthaccess.search_data(
            short_name='MODISA_L2_OC',
            temporal=(date.strftime('%Y-%m-%d'), date.strftime('%Y-%m-%d')),
            bounding_box=bbox
        )
        return granules
    except Exception as e:
        print(f"Search error for {date}: {e}")
        return []

# Download MODIS granule if not already cached locally
def download_and_cache_granule(granule, raw_dir):
    granule_name = granule['umm']['GranuleUR']
    cached_file = raw_dir / f"{granule_name}.nc"

    if cached_file.exists():
        print(f" - using cached file: {cached_file.name[:50]}")
        return str(cached_file)

    try:
        print(f" - downloading granule...")
        files = earthaccess.download([granule], local_path=str(raw_dir))
        if files and len(files) > 0:
            downloaded_file = Path(files[0])
            if downloaded_file != cached_file:
                downloaded_file.rename(cached_file)
            return str(cached_file)
    except Exception as e:
        print(f" - download failed: {e}")

    return None

# Extract modalities (chlor_a, Rrs, PAR, etc.) from a MODIS L2 file
def extract_habnet_modalities_from_l2(file_path, spatial_bounds):
    try:
        with nc.Dataset(file_path, 'r') as ds:
            # Ensure correct group structure
            if 'geophysical_data' not in ds.groups or 'navigation_data' not in ds.groups:
                print(f" - required groups not found in {Path(file_path).name}")
                return None

            geo_data = ds.groups['geophysical_data']
            nav_data = ds.groups['navigation_data']

            # Coordinates
            lats = nav_data.variables['latitude'][:]
            lons = nav_data.variables['longitude'][:]

            # Filter for spatial window
            lat_mask = (lats >= spatial_bounds['lat_min']) & (lats <= spatial_bounds['lat_max'])
            lon_mask = (lons >= spatial_bounds['lon_min']) & (lons <= spatial_bounds['lon_max'])
            spatial_mask = lat_mask & lon_mask

            if not np.any(spatial_mask):
                print(" - no data within spatial bounds")
                return None

            result = {
                'lats': lats[spatial_mask],
                'lons': lons[spatial_mask],
                'modalities': {}
            }

            for modality in HABNET_MODALITIES:
                if modality in geo_data.variables:
                    mod_data = geo_data.variables[modality][:]
                    if mod_data.shape == lats.shape:
                        valid_data = mod_data[spatial_mask]

                        # Check for masked arrays or NaNs
                        if hasattr(valid_data, 'mask'):
                            valid_mask = ~valid_data.mask
                        else:
                            valid_mask = np.isfinite(valid_data)

                        # Custom filter for chlorophyll values
                        if modality == 'chlor_a':
                            valid_mask = valid_mask & (valid_data > 0) & (valid_data < 1000)

                        if np.any(valid_mask):
                            final_values = valid_data[valid_mask]
                            result['modalities'][modality] = final_values
                            print(f" - {modality}: {len(final_values)} values, "
                                  f"range = {np.min(final_values):.3f}–{np.max(final_values):.3f}")

            return result if result['modalities'] else None

    except Exception as e:
        print(f" - error reading {Path(file_path).name}: {e}")
        return None


In [ ]:
class OptimizedHABNetDatacubeGenerator:
    def __init__(self, raw_dir, processed_dir, config=DATACUBE_CONFIG):
        self.raw_dir = Path(raw_dir)
        self.processed_dir = Path(processed_dir)
        self.config = config
        self.search_cache = {}

    def check_existing_datacube(self, stable_event_id):
        output_file = self.processed_dir / f"habnet_datacube_{stable_event_id}.pkl"
        return output_file.exists()

    def cached_search_modis_data(self, date, spatial_bounds):
        cache_key = f"{date.strftime('%Y-%m-%d')}_{spatial_bounds['lat_min']:.2f}_{spatial_bounds['lon_min']:.2f}"
        if cache_key in self.search_cache:
            return self.search_cache[cache_key]
        granules = search_modis_l2_data(date, spatial_bounds)
        self.search_cache[cache_key] = granules
        return granules

    def generate_datacube_for_event(self, event_row):
        stable_event_id = event_row['STABLE_EVENT_ID']
        if self.check_existing_datacube(stable_event_id):
            print(f"\nDatacube for {stable_event_id} already exists. Skipping.")
            return self.processed_dir / f"habnet_datacube_{stable_event_id}.pkl"

        event_date = event_row['SAMPLE_DATE']
        event_lat = event_row['LATITUDE']
        event_lon = event_row['LONGITUDE']
        hab_label = event_row['HAB_EVENT']

        print(f"\nGenerating datacube for event: {stable_event_id}")
        print(f"Date: {event_date.strftime('%Y-%m-%d')}, Location: ({event_lat:.3f}, {event_lon:.3f})")
        print(f"HAB Event: {hab_label}")

        spatial_bounds = calculate_spatial_bounds(event_lat, event_lon, self.config['spatial_extent_km'])
        temporal_bounds = calculate_temporal_bounds(event_date, self.config['temporal_extent_days'] - 1)
        total_days = (temporal_bounds['end_date'] - temporal_bounds['start_date']).days

        cube_size = self.config['spatial_extent_km'] // self.config['spatial_resolution_km']
        datacube_shape = (cube_size, cube_size, self.config['temporal_extent_days'])

        modality_datacubes = {
            modality: np.full(datacube_shape, np.nan) for modality in HABNET_MODALITIES
        }

        successful_days = 0

        for day_idx in range(total_days):
            current_date = temporal_bounds['start_date'] + timedelta(days=day_idx)
            print(f"  Day {day_idx+1}/{self.config['temporal_extent_days']} ({current_date.strftime('%Y-%m-%d')}):", end=" ")

            granules = self.cached_search_modis_data(current_date, spatial_bounds)
            if not granules:
                print("No data found")
                continue

            file_path = download_and_cache_granule(granules[0], self.raw_dir)
            if not file_path:
                print("Download failed")
                continue

            daily_data = extract_habnet_modalities_from_l2(file_path, spatial_bounds)
            if daily_data and daily_data['modalities']:
                for modality, values in daily_data['modalities'].items():
                    if len(values) > 0:
                        gridded = self._reproject_to_regular_grid(
                            daily_data['lats'], daily_data['lons'], values, spatial_bounds
                        )
                        modality_datacubes[modality][:, :, day_idx] = gridded
                successful_days += 1
                print(f"SUCCESS: {len(daily_data['modalities'])} modalities processed")
            else:
                print("No valid modality data")

        completeness = successful_days / self.config['temporal_extent_days']
        print(f"\nDatacube completeness: {successful_days}/{self.config['temporal_extent_days']} days ({completeness:.1%})")

        if completeness >= 0.8:
            datacube_data = {
                'datacubes': modality_datacubes,
                'metadata': {
                    'stable_event_id': stable_event_id,
                    'date': event_date,
                    'lat': event_lat,
                    'lon': event_lon,
                    'hab_label': hab_label,
                    'cell_count': event_row['CELLCOUNT'],
                    'spatial_bounds': spatial_bounds,
                    'temporal_bounds': temporal_bounds,
                    'config': self.config,
                    'data_completeness': completeness,
                    'successful_days': successful_days,
                    'modalities': list(modality_datacubes.keys()),
                    'generation_date': datetime.now()
                }
            }
            output_file = self.processed_dir / f"habnet_datacube_{stable_event_id}.pkl"
            with open(output_file, 'wb') as f:
                pickle.dump(datacube_data, f)
            print(f"Saved datacube to: {output_file}")
            return output_file
        else:
            print("Incomplete data – not saving.")
            return None

    def _reproject_to_regular_grid(self, lats, lons, values, spatial_bounds):
        grid_size = self.config['spatial_extent_km'] // self.config['spatial_resolution_km']
        target_lats = np.linspace(spatial_bounds['lat_min'], spatial_bounds['lat_max'], grid_size)
        target_lons = np.linspace(spatial_bounds['lon_min'], spatial_bounds['lon_max'], grid_size)
        lon_grid, lat_grid = np.meshgrid(target_lons, target_lats)

        source_pts = np.column_stack([lons.ravel(), lats.ravel()])
        target_pts = np.column_stack([lon_grid.ravel(), lat_grid.ravel()])
        valid_mask = np.isfinite(values)

        if np.sum(valid_mask) < 4:
            return np.full((grid_size, grid_size), np.nan)

        valid_pts = source_pts[:len(values)][valid_mask]
        valid_vals = values[valid_mask]

        try:
            interpolated = griddata(valid_pts, valid_vals, target_pts, method='linear', fill_value=np.nan)
            if np.any(np.isnan(interpolated)) and len(valid_vals) >= 1:
                interpolated_nn = griddata(valid_pts, valid_vals, target_pts, method='nearest', fill_value=np.nan)
                interpolated[np.isnan(interpolated)] = interpolated_nn[np.isnan(interpolated)]
            return interpolated.reshape(lon_grid.shape)
        except Exception as e:
            print(f"Interpolation failed: {e}")
            return np.full((grid_size, grid_size), np.nan)


In [ ]:
# nasa earth auth
def setup_nasa_earthdata():
    print("Setting up NASA Earthdata auth")
    try:
        auth = earthaccess.login()
        if auth:
            print(" - NASA Earthdata auth successful!")
            return True
        else:
            print(" - Auth failed")
            return False
    except Exception as e:
        print(f"  Authentication error: {e}")
        return False

# main pipeline runner
def run_datacube_generation():
    print("HAB Detection Datacube Pipeline (100km x 10 days)")
    print("=" * 50)

    # Step 1: Directory setup
    base_dir, raw_dir, processed_dir = setup_directories()
    print(f"Data directories created at: {base_dir.resolve()}")

    # Step 2: NASA Earthdata login
    if not setup_nasa_earthdata():
        print("NASA Earthdata login failed. Exiting...")
        return None

    # Step 3: Load ground-truth events
    try:
        hab_events = load_and_filter_hab_events()
    except FileNotFoundError:
        print("Missing 'habsos_20240430.csv'. Please ensure it is present.")
        return None

    # Step 4: Create MVP subset (1000 balanced)
    mvp_events = create_mvp_sample(hab_events, n_events=1000)
    mvp_file = base_dir / 'mvp_events.csv'
    mvp_events.to_csv(mvp_file, index=False)
    print(f"MVP events saved to {mvp_file.name}")

    # Step 5: Initialize datacube generator
    generator = OptimizedHABNetDatacubeGenerator(raw_dir, processed_dir)

    print("\nDATACUBE CONFIGURATION")
    print(f"Spatial Extent: {generator.config['spatial_extent_km']} km")
    print(f"Temporal Extent: {generator.config['temporal_extent_days']} days")
    print(f"Spatial Resolution: {generator.config['spatial_resolution_km']} km")
    print(f"Modalities: {HABNET_MODALITIES}")

    print("\nGenerating datacubes for MVP events")
    print("=" * 60)

    # Step 6: Process each event
    results = []
    start_time = datetime.now()

    for idx, (event_idx, event) in enumerate(mvp_events.iterrows()):
        print(f"\n{'='*50}")
        print(f"Processing event {idx+1}/{len(mvp_events)} - ID: {event_idx}")

        event_start = datetime.now()
        try:
            output_file = generator.generate_datacube_for_event(event)
            event_time = (datetime.now() - event_start).total_seconds() / 60

            results.append({
                'event_id': event_idx,
                'hab_label': event['HAB_EVENT'],
                'output_file': output_file,
                'success': output_file is not None,
                'processing_time_min': event_time
            })

            print(f"Event completed in {event_time:.1f} minutes")

        except Exception as e:
            event_time = (datetime.now() - event_start).total_seconds() / 60
            print(f"Failed to process event {event_idx}: {e}")
            results.append({
                'event_id': event_idx,
                'hab_label': event['HAB_EVENT'],
                'output_file': None,
                'success': False,
                'processing_time_min': event_time,
                'error': str(e)
            })

    # Step 7: Final Summary
    total_time = (datetime.now() - start_time).total_seconds() / 60
    success_count = sum([r['success'] for r in results])

    print("\n" + "=" * 60)
    print("DATACUBE GENERATION COMPLETE")
    print(f"Total time: {total_time:.1f} minutes")
    print(f"Average per event: {total_time / len(results):.1f} minutes")
    print(f"Events processed: {len(results)}")
    print(f"Successful: {success_count}")
    print(f"Failed: {len(results) - success_count}")

    if success_count > 0:
        print(f"\nDatacubes saved in: {processed_dir.resolve()}")

    return results, base_dir


# Entry point if running as script
if __name__ == "__main__":
    print("Starting HABNet Final Pipeline")
    print(f"Configuration: {DATACUBE_CONFIG}")
    print(f"Modalities: {HABNET_MODALITIES}")

    results, data_dir = run_datacube_generation()


Starting HABNet Final Pipeline
Configuration: {'spatial_extent_km': 100, 'temporal_extent_days': 10, 'spatial_resolution_km': 1, 'temporal_resolution_days': 1}
Modalities: ['chlor_a', 'Rrs_412', 'Rrs_443', 'Rrs_488', 'Rrs_531', 'Rrs_555', 'par']
HAB Detection Datacube Pipeline (100km x 10 days)
Data directories created at: /content/habnet_data
Setting up NASA Earthdata auth
 - NASA Earthdata auth successful!
Loading and filtering ground truth HAB events...
Total filtered HAB event candidates: 86,160
 - HAB events: 8,736
 - Non-HAB events: 77,424
 - Date range: 2003-01-02 to 2018-12-31

Sampling 1000 MVP events...
Sample complete: 1000 events
 - Positive (HAB): 500
 - Negative (non-HAB): 500
 - Range: 2015-01-18 to 2018-12-17
MVP events saved to mvp_events.csv

DATACUBE CONFIGURATION
Spatial Extent: 100 km
Temporal Extent: 10 days
Spatial Resolution: 1 km
Modalities: ['chlor_a', 'Rrs_412', 'Rrs_443', 'Rrs_488', 'Rrs_531', 'Rrs_555', 'par']

Generating datacubes for MVP events

Processin

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

 - chlor_a: 566 values, range = 0.113–7.604
 - Rrs_412: 566 values, range = 0.003–0.019
 - Rrs_443: 566 values, range = 0.003–0.022
 - Rrs_488: 566 values, range = 0.003–0.027
 - Rrs_531: 566 values, range = 0.001–0.028
 - Rrs_555: 566 values, range = -0.001–0.026
 - par: 2408 values, range = 6.390–35.736
SUCCESS: 7 modalities processed
  Day 2/10 (2015-01-10):  - downloading granule...


...

 - chlor_a: 1352 values, range = 0.493–82.628
 - Rrs_412: 1353 values, range = 0.001–0.008
 - Rrs_443: 1353 values, range = 0.000–0.008
 - Rrs_488: 1353 values, range = 0.000–0.011
 - Rrs_531: 1353 values, range = 0.002–0.016
 - Rrs_555: 1353 values, range = 0.002–0.017
 - par: 2950 values, range = 13.560–33.162
SUCCESS: 7 modalities processed

Datacube completeness: 10/10 days (100.0%)
Saved datacube to: habnet_data/processed_datacubes/habnet_datacube_20151130_27.9604_-82.8243_0.pkl
Event completed in 0.6 minutes

Processing event 151/1000 - ID: 150

Generating datacube for event: 20151130_27.311_-82.5772_290000
Date: 2015-11-30, Location: (27.311, -82.577)
HAB Event: 1
  Day 1/10 (2015-11-21):  - using cached file: MODISA_L2_OC_AQUA_MODIS.20151121T182501.L2.OC.nc_2
 - par: 3252 values, range = 0.314–21.422
SUCCESS: 1 modalities processed
  Day 2/10 (2015-11-22):  - using cached file: MODISA_L2_OC_AQUA_MODIS.20151122T191001.L2.OC.nc_2
 - chlor_a: 760 values, range = 0.138–77.021
 - Rr

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

 - par: 5734 values, range = 15.396–30.738
SUCCESS: 1 modalities processed

Datacube completeness: 10/10 days (100.0%)
Saved datacube to: habnet_data/processed_datacubes/habnet_datacube_20151201_30.242_-87.6761_620000.pkl
Event completed in 0.5 minutes

Processing event 157/1000 - ID: 156

Generating datacube for event: 20151201_30.273_-87.5509_1000000
Date: 2015-12-01, Location: (30.273, -87.551)
HAB Event: 1
  Day 1/10 (2015-11-22):  - using cached file: MODISA_L2_OC_AQUA_MODIS.20151122T191001.L2.OC.nc_2
 - chlor_a: 4443 values, range = 0.485–84.770
 - Rrs_412: 4562 values, range = -0.010–0.004
 - Rrs_443: 4562 values, range = -0.007–0.005
 - Rrs_488: 4562 values, range = -0.004–0.009
 - Rrs_531: 4562 values, range = -0.002–0.016
 - Rrs_555: 4562 values, range = -0.002–0.018
 - par: 5060 values, range = 28.790–32.410
SUCCESS: 7 modalities processed
  Day 2/10 (2015-11-23):  - using cached file: MODISA_L2_OC_AQUA_MODIS.20151123T181501.L2.OC.nc_2
 - chlor_a: 779 values, range = 0.392–8